# Cypher queries

## Testing import from blocks and transaction topics by sample queries

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

uri = "neo4j://localhost:7687"
credentialsNeo4j = pd.read_json('credentialsNeo4j.json')
location = "local"
username = credentialsNeo4j[location][0]['user']
password = credentialsNeo4j[location][0]['pwd']
driver = GraphDatabase.driver(uri, auth=(username, password))

## Conversion functions 

In [ ]:
#### Convert final query by cutting all linebreaks...
def createCypherForEvents(query):
    new_query = query.replace('$', 'event.')
    return new_query

#### Convert final query by cutting all linebreaks...
def omitSpacings(query):
    omit_characters = ["\n", "     ", "     ", "  "]
    for i in omit_characters:
        query = query.replace(i, " ")
    return query

## Example Data

In [ ]:
#test data 
test = {'txid': '88d35b8d35ccaf8a315a2274d83081ab62aeda06fb2029af5c65b2a858f22fcb',
 'block_hash': '00000000000000000005278b3b0d99196ded30c92edb64181abab711fb47527b',
 'block_date': '2021-05-23',
 'outDegree': 3,
 'inDegree': 1,
 'outSum': 656215465,
 'inSum': 656215465,
 'input_list': [{'addr': 'coinbase', 'val': 656215465}],
 'output_list': [{'addr': '1Bf9sZvBHPFGVPX71WX2njhd1NXKv5y7v5', 'outNr': 0, 'val': 5}]}

## Transactions


In [ ]:
###Address matching Structure + amendmends

transactionQuery = '''
    MERGE (t:Transaction{txid: $txid}) 
        SET t += {inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, inSum: $inSum, date: date($date_block)}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in $output_list | 
        MERGE (o_a:Address{address: output.addr}) 
            ON CREATE SET o_a += {inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_a:Address{address: input.addr})
            ON CREATE SET i_a += {inDegree: 0, outDegree: 1}
            ON MATCH SET  i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:Sends{value: input.val}]->(t))
    '''


def run_tx(tx, event):
    tx.run(transactionQuery, 
           txid         = event["txid"], 
           output_list  = event["output_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           inSum        = event["inSum"],
           input_list   = event["input_list"], 
           date_block   = event["block_date"])


In [ ]:
t = test
with driver.session() as session:
    session.write_transaction(run_tx, t)

## Final TX query

In [ ]:
#### Convert final query by cutting all linebreaks...
transactionQuery_forEvents = createCypherForEvents(transactionQuery)
print(transactionQuery_forEvents)

In [ ]:
#### Convert final query by cutting all linebreaks...
transactionQuery_final = omitSpacings(transactionQuery_forEvents)
print(transactionQuery_final)

## Blocks
!!block_median_time

In [ ]:
## !need to amend!
blockQuery = '''
MATCH (p:Block) WHERE p.hash = $p_block_hash 
MERGE (b:Block {hash: $block_hash}) 
    SET b += {hash: $block_hash, height: $block_height, blockDate: date($block_date), mediantime: datetime($block_timestamp) }
MERGE (p)-[r:Precedes]->(b)
'''

def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run(query, block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

In [ ]:
block = b3
with driver.session() as session:
    session.write_transaction(create_block, 
                              block['block_hash'], 
                              block['block_height'], 
                              block['block_timestamp'],
                              block['p_block_hash'])
    

## Final Block query

In [ ]:
#### Convert final query by cutting all linebreaks...
blockQuery_forEvents = createCypherForEvents(blockQuery)
print(blockQuery_forEvents)

In [ ]:
#### Convert final query by cutting all linebreaks...
blockQuery_final = omitSpacings(blockQuery_forEvents)
print(blockQuery_final)